In [1]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import math
import pandas as pd 


In [2]:
def haversine(coord1, coord2):
    lat1, lon1 = [float(i) for i in coord1]
    lat2, lon2 = [float(i) for i in coord2]
    rad=math.pi/180
    dlat= lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [3]:
def get_geocoder(url_location): # gets geographical lat/long coordinates
        try:
            #print(url_location)
            coords = re.search(r"!3d-?\d\d?\.\d{4,8}!4d-?\d\d?\.\d{4,8}",
                            url_location).group()
            coord = coords.split('!3d')[1]
            return tuple(coord.split('!4d'))
        except (TypeError, AttributeError):
            return ("", "")

def scroll_the_page(i):
    try:
        #section_loading = self.driver.find_element_by_class_name("section-loading")
        section_loading = driver.find_element(By.CLASS_NAME, "section-loading")
        while True:
            # if i >= len(self.driver.find_elements(By.CLASS_NAME, "place-result-container-place-link")):
            if i >= len(driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a')):
                actions = ActionChains(driver)
                actions.move_to_element(section_loading).perform()
                time.sleep(2)
            else:
                break
    except:
        pass


In [4]:
def Busqueda_20(string_busqueda):
    PATH = '/home/pomilusky/Escriptori/Ironhack/chromedriver'
    query = string_busqueda
    a = dict({})
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")
    options.add_argument("no_sandbox")
    driver = webdriver.Chrome(PATH, options=options)
    driver.get("https://www.google.es/maps/search/"+query.replace(" ", "+")+"/")
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div/div/button/span').click()
    n = int(driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/span/span[2]').text)
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a').click()
    name= driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
    time.sleep(5)
    a[name+'_0']= get_geocoder(driver.current_url)
    # print(driver.current_url)
    for i in range(2,n+1):
        try:
            # driver.find_element_by_xpath('//*[@id="omnibox-singlebox"]/div[1]/div[1]/button').click()
            driver.implicitly_wait(5)
            # scroll_the_page(2)
            name= driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
            driver.implicitly_wait(5)
            driver.find_element(By.XPATH,f'//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/div[2]/div[2]/div[{i}]/div[3]/div[1]/div[1]/span[1]').click()
            time.sleep(3)
            a[name+f'_{i}']= get_geocoder(driver.current_url)
            try:
                if i%3 == 0:
                    driver.find_element(By.XPATH,'//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/button[2]/div').click()
                    time.sleep(2)
            except:
                pass

        except:
            print(i)
    driver.quit()
    # print(a)
    return a
    # print
    
    


In [32]:
# Ara em faré una funció que em vagi a una localització i em trobi l'element més proper del tipus que li demani.
def NearSearch(element,coords, n=5):
    if n>20: n = 20
    PATH = '/home/pomilusky/Escriptori/Ironhack/chromedriver'
    query = element.replace(" ", "+")
    a = dict({})
    lat, lon = coords
    options = webdriver.ChromeOptions()
    # options.add_argument("headless")
    options.add_argument("no_sandbox")
    driver = webdriver.Chrome(PATH, options=options)
    driver.get(f"https://www.google.com/maps/search/{query}/@{lat},{lon},12z")
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div/div/button/span').click()
    driver.implicitly_wait(5)
    driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a').click()
    name= driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
    time.sleep(5)
    crds = get_geocoder(driver.current_url)
    dist = haversine(coords, crds)
    
    a[1]= {'nombre':name,'location':{'type': 'Point', 'coordinates':list(crds)}, 'distance': dist, 'Entidad': element}

    # print(driver.current_url)
    for i in range(2,n+1):
        try:
            # driver.find_element_by_xpath('//*[@id="omnibox-singlebox"]/div[1]/div[1]/button').click()
            driver.implicitly_wait(5)
            # scroll_the_page(2)
            name= driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
            driver.implicitly_wait(5)
            driver.find_element(By.XPATH,f'//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/div[2]/div[2]/div[{i}]/div[3]/div[1]/div[1]/span[1]').click()
            time.sleep(3)
            crds = get_geocoder(driver.current_url)
            dist = haversine(coords, crds)
            a[i]= {'nombre':name,'location':{'type': 'Point', 'coordinates':list(crds)}, 'distance': dist, 'Entidad': element}

            try:
                if i%3 == 0:
                    driver.find_element(By.XPATH,'//*[@id="sGi9mc-m5SR9c-bottom-pane"]/div/div[1]/div/div/div/button[2]/div').click()
                    time.sleep(2)
            except:
                pass

        except:
            print(i)
    driver.quit()
    # print(a)
    return pd.DataFrame(dict(sorted(a.items() , key = lambda x: x[1]['distance']))).T

In [83]:
df = pd.read_csv('../Data/castilla_leon/toMongo.csv', index_col='Unnamed: 0')

In [84]:
df[['Entidad Singular de Población','location']].head()

,Entidad Singular de Población,location
0,ADANERO,"{'type': 'Point', 'coordinates': [4.6066666666..."
1,ESTACIÓN (LA),"{'type': 'Point', 'coordinates': [0.0, 0.0]}"
2,ADRADA (LA),"{'type': 'Point', 'coordinates': [4.635, 40.29..."
3,ALBORNOS,"{'type': 'Point', 'coordinates': [4.8813888888..."
4,ALDEANUEVA DE SANTA CRUZ,"{'type': 'Point', 'coordinates': [5.4194444444..."


In [90]:
i = 5
list(df.location.sample(20))[i] , list(df['Entidad Singular de Población'].sample(20))[i]

("{'type': 'Point', 'coordinates': [3.8397222222222, 41.363055555556]}",
 'BASARDILLA')

In [93]:
to_scrap = dict({'TORRALBA DEL MORAL':(42.326388888889,-6.5072222222222),
'SANTA GADEA':(41.462777777778,2.0191666666667),
'RINCONADA':(40.6779,-4.9445694444444),
'SAN CIPRIANO DE RUEDA':(41.531666666667,-3.1566666666667),
'BASARDILLA': (41.363055555556, 3.8397222222222)})

In [96]:
for k,v in to_scrap.items():
    print(k, v)
    things_2_look_4 = ['supermercado','polideportivo', 'hospital','centro de salud', 'gasolinera']
    lista_df = []
    for i in things_2_look_4:
        df = NearSearch(i,v)
        lista_df.append(df)
    new_df = pd.concat(lista_df, axis=0, join="outer", ignore_index=True, keys=None, levels=None, names=None, verify_integrity=False, copy=False)
    a = k.replace(' ','_')
    new_df.to_csv(f'../Data/castilla_leon/{a}.csv')

SyntaxError: invalid syntax (2715469134.py, line 9)

In [37]:
new_df

,nombre,location,distance,Entidad
0,Supermercado Udaco Coto de Puenteviejo,"{'type': 'Point', 'coordinates': ['40.9944', '...",5.458863,supermercado
1,Unide,"{'type': 'Point', 'coordinates': ['40.8935949'...",6.190699,supermercado
2,Unide,"{'type': 'Point', 'coordinates': ['40.860005',...",10.256213,supermercado
3,Supermercados La Plaza de Dia,"{'type': 'Point', 'coordinates': ['41.05777', ...",15.488754,supermercado
4,Supermercado Unide,"{'type': 'Point', 'coordinates': ['41.056777',...",15.519871,supermercado
5,Polideportivo y frontón de El Bohodón,"{'type': 'Point', 'coordinates': ['40.9265064'...",3.46567,polideportivo
6,Polideportivo Tomás Luis de Victoria,"{'type': 'Point', 'coordinates': ['40.9263627'...",3.522632,polideportivo
7,Polideportivo Tomás Luis de Victoria,"{'type': 'Point', 'coordinates': ['40.8885901'...",6.625333,polideportivo
8,Frontón Municipal,"{'type': 'Point', 'coordinates': ['40.9350408'...",10.361044,polideportivo
9,Polideportivo,"{'type': 'Point', 'coordinates': ['40.9155574'...",11.037178,polideportivo


In [77]:

pd.DataFrame(a)

,coords,distance
Mercadona_0,"(40.9369507, -4.1119966)",7.366464
Mercadona_2,"(40.958406, -4.10454)",7.974593
Mercadona_3,"(40.9296782, -4.0981471)",8.669786
Mercadona_4,"(40.6864149, -3.966313)",35.04361
Mercadona_7,"(40.6525655, -4.007391)",36.581731
Mercadona_5,"(40.636121, -4.0165501)",37.915633
Mercadona_8,"(40.6021469, -4.058892)",40.218796
Mercadona_9,"(40.5838053, -3.9968267)",43.919941
Mercadona_12,"(40.5638206, -4.0134942)",45.478659
Mercadona_6,"(40.5710476, -3.9244568)",47.856995


In [31]:
mec = Busqueda_20('Hospitales Zamora')
mec

/tmp/ipykernel_32016/2228515432.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH, options=options)


{'Complejo Hospitalario de Zamora_0': ('41.5106075', '-5.7414055'), 'Complejo Hospitalario de Zamora_2': ('41.5172718', '-5.7469789'), 'Hospital Recoletas Zamora_3': ('41.5144616', '-5.7313506'), 'Public Welfare Complex De Zamora_4': ('41.511571', '-5.7279945'), 'Provincial Hospital Zamora_5': ('41.5121636', '-5.7291344'), 'Unidad de Oncologia Radioterapica_6': ('41.5152042', '-5.7290327'), 'Avenida De requejo_7': ('41.5101073', '-5.7418573'), 'Centro Médico Recoletas Tres Cruces_8': ('41.5055606', '-5.7409155'), 'Centro Médico Recoletas VEA_9': ('41.5175655', '-5.7474751'), 'Parking Hospital Recoletas Zamora_10': ('41.5132504', '-5.732231'), 'Centro Salud Virgen de la Concha_11': ('41.5108832', '-5.7273544'), 'Aparcamiento Hospital Provincial de Zamora_12': ('41.51087', '-5.7414487'), 'Resonance Pet Servicios de Gestión A.I.e._13': ('41.5161758', '-5.7517713'), 'Centro de Salud Zamora Norte_14': ('41.5113515', '-5.7399986'), 'Centro de Salud Zamora Sur_15': ('41.5168588', '-5.7464349'

{'Complejo Hospitalario de Zamora_0': ('41.5106075', '-5.7414055'),
 'Complejo Hospitalario de Zamora_2': ('41.5172718', '-5.7469789'),
 'Hospital Recoletas Zamora_3': ('41.5144616', '-5.7313506'),
 'Public Welfare Complex De Zamora_4': ('41.511571', '-5.7279945'),
 'Provincial Hospital Zamora_5': ('41.5121636', '-5.7291344'),
 'Unidad de Oncologia Radioterapica_6': ('41.5152042', '-5.7290327'),
 'Avenida De requejo_7': ('41.5101073', '-5.7418573'),
 'Centro Médico Recoletas Tres Cruces_8': ('41.5055606', '-5.7409155'),
 'Centro Médico Recoletas VEA_9': ('41.5175655', '-5.7474751'),
 'Parking Hospital Recoletas Zamora_10': ('41.5132504', '-5.732231'),
 'Centro Salud Virgen de la Concha_11': ('41.5108832', '-5.7273544'),
 'Aparcamiento Hospital Provincial de Zamora_12': ('41.51087', '-5.7414487'),
 'Resonance Pet Servicios de Gestión A.I.e._13': ('41.5161758', '-5.7517713'),
 'Centro de Salud Zamora Norte_14': ('41.5113515', '-5.7399986'),
 'Centro de Salud Zamora Sur_15': ('41.5168588'

{'Hospital Universitario de León_0': ('42.6188257', '-5.5724149'),
 'Hospital Universitario de León_2': ('42.6393236', '-5.5940199'),
 'Hospital Monte San Isidro_3': ('41.6556947', '-4.7196729'),
 'University Clinical Hospital of Valladolid_4': ('41.295331', '-4.9114143'),
 'Medina del Campo Hospital_5': ('41.6294871', '-4.7134255'),
 'Rio Hortega University Hospital_6': ('42.6197426', '-5.5725965'),
 'Hospital Universitario de León Urgencias_7': ('42.0041234', '-5.6778091'),
 'Hospital Comarcal de Benavente_8': ('42.57592', '-5.5450522'),
 'Hospital Santa Isabel_9': ('40.9426964', '-4.1286328'),
 'Complejo Asistencial de Segovia_10': ('42.361942', '-3.6848983'),
 'Burgos University Hospital_11': ('42.6698273', '-2.9888184'),
 'Hospital Santiago Apóstol_12': ('40.6501965', '-4.6924905'),
 'Hospital Provincial_13': ('40.6658877', '-4.6666992'),
 'Hospital Nuestra Señora de Sonsoles_14': ('41.5144616', '-5.7313506'),
 'Public Welfare Complex De Zamora_15': ('40.9640946', '-5.6732785'),
 

In [7]:
driver = webdriver.Chrome(executable_path='/home/pomilusky/Escriptori/Ironhack/chromedriver')

/tmp/ipykernel_6191/235964592.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/home/pomilusky/Escriptori/Ironhack/chromedriver')
